# Eat Safe, Love

## Notebook Set Up

In [65]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

mongoimport --db uk_food --collection establishments --file "..\Resources\establishments.json" --jsonArray

In [66]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [84]:
# Confirm database loaded properly
mongo.list_database_names()

['admin',
 'classDB',
 'config',
 'local',
 'met',
 'petsitly_marketing',
 'travel_db',
 'uk_food']

In [67]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [68]:
# review the collections in our database
collections = db.list_collection_names()
for collection in collections: 
    pprint(collection)

'establishments'


In [85]:
# Find and display one document using find_one
document = establishments.find_one()

# Display the document using pprint
pprint(document)


{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64adf331255015fae8d386c7'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [86]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [87]:
# Find the establishments with a hygiene score of 20
query = {"hygiene_score": 20}
results = establishments.find(query)

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print("Number of establishments with hygiene score 20:", count)

# Check if the cursor has any items before accessing the first item
if count > 0:
    # Display the first document in the results using pprint
    print(result[0])
else:
    print("No establishments found with hygiene score 20.")


Number of establishments with hygiene score 20: 0
No establishments found with hygiene score 20.


In [88]:
# Convert the result to a Pandas DataFrame
import pandas as pd
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
num_rows = len(df)
print("Number of rows:", num_rows)

# Display the first 10 rows of the DataFrame
print(df.head(10))

Number of rows: 0
Empty DataFrame
Columns: []
Index: []


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [89]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.

# Create the query
query = {"LocalAuthorityName": "London", "RatingValue": {"$gte": 4}}

# Find the establishments with the query
results = establishments.find(query)

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print("Number of establishments with London as the Local Authority and RatingValue >= 4:", count)

# Check if the cursor has any documents
if count > 0:
    # Display the first document in the results using pprint
    pprint(results[0])
else:
    print("No documents found")


Number of establishments with London as the Local Authority and RatingValue >= 4: 0
No documents found


In [79]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
num_rows = len(df)
print("Number of rows:", num_rows)

# Display the first 10 rows of the DataFrame
print(df.head(10))

Number of rows: 0
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [93]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

# Set the search range
degree_search = 0.01

# Set the latitude and longitude for the new restaurant "Penang Flavours"
latitude = 123.456  # Replace with the latitude of "Penang Flavours"
longitude = 789.012  # Replace with the longitude of "Penang Flavours"

# Define the query and sort parameters
query = {
    "RatingValue": 5,
    "latitude": {
        "$gte": latitude - degree_search,
        "$lte": latitude + degree_search
    },
    "longitude": {
        "$gte": longitude - degree_search,
        "$lte": longitude + degree_search
    }
}
sort = [("hygiene_score", 1)]

# Find the top 5 establishments matching the criteria and sorted by hygiene score
results = establishments.find(query).sort(sort).limit(5)

# Print the results
for result in results:
    pprint(result)

In [96]:
# Convert result to Pandas DataFrame

# Set the latitude and longitude for the new restaurant "Penang Flavours"
latitude = 123.456  # Replace with the latitude of "Penang Flavours"
longitude = 789.012  # Replace with the longitude of "Penang Flavours"

# Set the search range
degree_search = 0.01

# Define the query and sort parameters
query = {
    "RatingValue": 5,
    "latitude": {
        "$gte": latitude - degree_search,
        "$lte": latitude + degree_search
    },
    "longitude": {
        "$gte": longitude - degree_search,
        "$lte": longitude + degree_search
    }
}
sort = [("hygiene_score", 1)]

# Find the top 5 establishments matching the criteria and sorted by hygiene score
results = establishments.find(query).sort(sort).limit(5)

# Convert the results to a Pandas DataFrame
df = pd.DataFrame(list(results))

# Display the DataFrame
print(df)

Empty DataFrame
Columns: []
Index: []


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [99]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results

# Create the pipeline stages
pipeline = [
    {"$match": {"hygiene_score": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]

# Execute the pipeline
results = establishments.aggregate(pipeline)

# Count the number of documents in the result
num_documents = establishments.count_documents({"hygiene_score": 0})

# Print the number of documents in the result
print("Number of documents:", num_documents)

# Print the first 10 results
counter = 0
for result in results:
    print(result)
    counter += 1
    if counter >= 10:
        break


Number of documents: 0


In [100]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
num_rows = len(df)
print("Number of rows:", num_rows)

# Display the first 10 rows of the DataFrame
print(df.head(10))

Number of rows: 0
Empty DataFrame
Columns: []
Index: []
